## Integrate radially first

Important: Hyperspy file needs to be calibrated and distortion corrected and beam centered

In [1]:
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
%matplotlib qt
import pyxem as pxm
import hyperspy.api as hs
import os, glob, tqdm
import gc
import numpy as np
from diffsims.utils.sim_utils import get_electron_wavelength

In [2]:
root = r"G:\My Drive\PhD\projects\sed_ML\clustering_poyen"
#experiment = r"data\20221029_first_attempt\cluster_navigators"

dp_data = r"data\experimental\tcdh_thin_20211117_182814\20211117_182814_data_calibrated_data.hspy"
dp_data = r"data\experimental\polytype_all_20220720_155557\20220720_155557_data_calibrated_data.hspy"
dp_data = r"data\experimental\dcdh_20220628_114631\20220628_114631_data_calibrated_data.hspy"
dp_data = r"data\experimental\tcdh_thin_20211117_182521\20211117_182521_data_calibrated_data.hspy"

p = os.path.join(root, dp_data)
dp = hs.load(p, lazy=True)
#dp = dp.inav[67:107,52:87]
dp

<LazyElectronDiffraction2D, title: , dimensions: (255, 255|515, 515)>

In [3]:
dp = dp.isig[100:-100, 100:-100]
dp

<LazyElectronDiffraction2D, title: , dimensions: (255, 255|315, 315)>

In [4]:
dp.compute()

[########################################] | 100% Completed | 229.93 s


In [5]:
#dp.plot(vmax=10)

In [6]:
from pyxem.detectors import Medipix515x515Detector

# Simulation microscope values (for azimuthal integration)
beam_energy = 200.0  #keV
detector_pix_size = 55e-6  #m


print(p)

calibration = dp.axes_manager.signal_axes[0].scale
detector_size = dp.axes_manager.signal_axes[0].size
radial_steps = int(np.ceil((int(detector_size / 2) - 1) / 2) * 2)

# # Old way (pyxem way):
#dp.set_diffraction_calibration(calibration)
dp.metadata.Signal.ai = None
dp.unit = "k_A^-1"
dp.set_experimental_parameters(beam_energy=beam_energy)
dp.set_ai(center=([detector_size / 2, detector_size / 2]))

# New way (pyFAI)
# detector = Medipix515x515Detector()
# wavelength = get_electron_wavelength(beam_energy) * 1e-10
# camera_length = detector_pix_size / (wavelength * calibration * 1e10)
# center = ([detector_size / 2, detector_size / 2])
# unit = "k_A^-1"
# dp.unit = unit
# ai = AzimuthalIntegrator(dist=camera_length, detector=detector, wavelength=wavelength)
# ai.setFit2D(directDist=camera_length * 1000, centerX=center[1], centerY=center[0])
# dp.metadata.set_item("Signal.ai", ai)

radial = dp.get_azimuthal_integral1d(npt=radial_steps)
print(radial)
name = "{}_radial.hspy".format(os.path.basename(p).split('.')[0])
radial.save(os.path.join(os.path.dirname(p), name), overwrite=True)

radial2d = dp.get_azimuthal_integral2d(npt=radial_steps)
name = "{}_radial2d.hspy".format(os.path.basename(p).split('.')[0])
radial2d.save(os.path.join(os.path.dirname(p), name), overwrite=True)
print(f"Finished with {name}")

del dp
gc.collect()


G:\My Drive\PhD\projects\sed_ML\clustering_poyen\data\experimental\tcdh_thin_20211117_182521\20211117_182521_data_calibrated_data.hspy


[########################################] | 100% Completed | 431.99 s
<ElectronDiffraction1D, title: , dimensions: (255, 255|156)>


ERROR:hyperspy.io_plugins._hierarchical:The writer could not write the following information in the file: ai : Detector Detector	 Spline= None	 PixelSize= 1.293e-04, 1.293e-04 m
Wavelength= 2.507934e-12m
SampleDetDist= 1.000000e+00m	PONI= 2.035828e-02, 2.035828e-02m	rot1=0.000000  rot2= 0.000000  rot3= 0.000000 rad
DirectBeamDist= 1000.000mm	Center: x=157.500, y=157.500 pix	Tilt=0.000 deg  tiltPlanRotation= 0.000 deg
Traceback (most recent call last):
  File "c:\Users\jf631\miniconda3\envs\pyxem_py4dstem\lib\site-packages\hyperspy\io_plugins\_hierarchical.py", line 786, in dict2group
    group.attrs[key] = value
  File "h5py\_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py\_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "c:\Users\jf631\miniconda3\envs\pyxem_py4dstem\lib\site-packages\h5py\_hl\attrs.py", line 103, in __setitem__
    self.create(name, data=value)
  File "c:\Users\jf631\miniconda3\envs\pyxem_py4dstem\lib\site-packages\h5py\_hl\attr

[########################################] | 100% Completed | 525.10 s


ERROR:hyperspy.io_plugins._hierarchical:The writer could not write the following information in the file: ai : Detector Detector	 Spline= None	 PixelSize= 1.293e-04, 1.293e-04 m
Wavelength= 2.507934e-12m
SampleDetDist= 1.000000e+00m	PONI= 2.035828e-02, 2.035828e-02m	rot1=0.000000  rot2= 0.000000  rot3= 0.000000 rad
DirectBeamDist= 1000.000mm	Center: x=157.500, y=157.500 pix	Tilt=0.000 deg  tiltPlanRotation= 0.000 deg
Traceback (most recent call last):
  File "c:\Users\jf631\miniconda3\envs\pyxem_py4dstem\lib\site-packages\hyperspy\io_plugins\_hierarchical.py", line 786, in dict2group
    group.attrs[key] = value
  File "h5py\_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py\_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "c:\Users\jf631\miniconda3\envs\pyxem_py4dstem\lib\site-packages\h5py\_hl\attrs.py", line 103, in __setitem__
    self.create(name, data=value)
  File "c:\Users\jf631\miniconda3\envs\pyxem_py4dstem\lib\site-packages\h5py\_hl\attr

Finished with 20211117_182521_data_calibrated_data_radial2d.hspy


2591

In [7]:
radial.plot()
radial2d.plot(vmax=10)

In [8]:
radial

<ElectronDiffraction1D, title: , dimensions: (255, 255|156)>

## Crop, interpolate (with rebin) to match simulated range

The simulated range has the following format. Make sure the processed exp data has the same:
- Pixel size: 147 px
- Range in q (no $2\pi/d$ but just $1/d$): (0.10777668889613681, 1.318191810345058) $\AA^{-1}$

In [9]:
# DO NOT CHANGE!!!
crop_range_q = (0.10777668889613681, 1.318191810345058) # A-1
crop_size = 147 #px

In [10]:
%matplotlib qt
import hyperspy.api as hs
import os, glob
import numpy as np
from scipy import interpolate
import warnings
import matplotlib.pyplot as plt

In [11]:
def interpolate_1d(signal_data, q_array, crop_range_q, crop_size):
    # Do interpolation
    x = q_array
    y = signal_data
    f = interpolate.interp1d(x, y, fill_value='extrapolate')

    # Generate new data
    x_new = np.linspace(crop_range_q[0], crop_range_q[1], crop_size)
    y_interpol = f(x_new)
    return y_interpol

def interpolate_2d(signal_data, q_array, crop_range_q, crop_size):
    signal_data = signal_data.T

    y_interpol_2d = \
        np.vstack([interpolate_1d(row, q_array, crop_range_q, crop_size)
                   for row in signal_data])
    return y_interpol_2d.T

## 1d interpolation

In [12]:
folder = os.path.dirname(p)
file_extension = '*.hspy'

path = os.path.join(root, folder, file_extension)
paths = glob.glob(path)
paths = [p for p in paths if 'radial.' in p and 'crop' not in p]
paths.sort()
paths

['G:\\My Drive\\PhD\\projects\\sed_ML\\clustering_poyen\\data\\experimental\\tcdh_thin_20211117_182521\\20211117_182521_data_calibrated_data_radial.hspy']

In [13]:
path = paths[0]
dp = hs.load(path)

q_exp = dp.axes_manager.signal_axes[0].axis
if q_exp.min() > crop_range_q[0] or q_exp.max() < crop_range_q[1]:
    warnings.warn("The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d")

q_array = dp.axes_manager.signal_axes[0].axis
q_new = np.linspace(crop_range_q[0], crop_range_q[1], crop_size)
dp_crop = dp.map(interpolate_1d, q_array = q_array, crop_range_q = crop_range_q, crop_size = crop_size,
                    show_progressbar=True, parallel=True, inplace=False)

# Correct for axes calibration
sig_ax = dp_crop.axes_manager.signal_axes[0]
sig_ax.offset = crop_range_q[0]
sig_ax.scale = (crop_range_q[1] - crop_range_q[0])/ crop_size
print(dp_crop)

# Save files
name = "{}_crop_interpolate.hspy".format(os.path.basename(path).split('.')[0])
dp_crop.save(os.path.join(os.path.dirname(path), name), overwrite=True)
name = "{}_crop_interpolate.npz".format(os.path.basename(path).split('.')[0])
np.savez(os.path.join(os.path.dirname(path), name), y=dp_crop.data, x=q_new)

# Save files not interpolated
# Non interpolated
dp_crop_no = dp.isig[crop_range_q[0]:crop_range_q[1]]
q_old = dp_crop_no.axes_manager.signal_axes[0].axis
name_no = "{}_crop.hspy".format(os.path.basename(path).split('.')[0])
dp_crop_no.save(os.path.join(os.path.dirname(path), name_no), overwrite=True)
name_no = "{}_crop.npz".format(os.path.basename(path).split('.')[0])
np.savez(os.path.join(os.path.dirname(path), name_no), y=dp_crop_no.data, x=q_old)

[                                        ] | 0% Completed | 115.96 ms

C:\Users\jf631\AppData\Local\Temp\ipykernel_3116\3279812516.py:6: UserWarning: The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d
  warnings.warn("The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d")


[########################################] | 100% Completed | 7.20 ss
<ElectronDiffraction1D, title: , dimensions: (255, 255|147)>


In [14]:
# Plot results
plt.plot(q_new, dp_crop.mean().data, label=name)
plt.plot(q_old, dp_crop_no.mean().data, label=name_no)

plt.xlabel("q vector (A-1)")
plt.ylabel("Summed dp intensity")
plt.legend()
plt.tight_layout()
name_fig = "{}_fig.png".format(os.path.basename(path).split('.')[0])
plt.savefig(os.path.join(os.path.dirname(path), name_fig))

## Interpolate 2D

In [15]:
folder = os.path.dirname(p)
file_extension = '*.hspy'

path = os.path.join(root, folder, file_extension)
paths = glob.glob(path)
paths_2d = [p for p in paths if 'radial2d.' in p and 'crop' not in p]
paths_2d.sort()
paths_2d

['G:\\My Drive\\PhD\\projects\\sed_ML\\clustering_poyen\\data\\experimental\\tcdh_thin_20211117_182521\\20211117_182521_data_calibrated_data_radial2d.hspy']

In [16]:
path = paths_2d[0]
dp2d = hs.load(path)

# Save files not interpolated
# Non interpolated
dp2d_crop_no = dp2d.isig[:,crop_range_q[0]:crop_range_q[1]]
q_old = dp2d_crop_no.axes_manager.signal_axes[1].axis
name_no = "{}_crop2d.hspy".format(os.path.basename(path).split('.')[0])
dp2d_crop_no.save(os.path.join(os.path.dirname(path), name_no), overwrite=True)
name_no = "{}_crop2d.npz".format(os.path.basename(path).split('.')[0])
np.savez(os.path.join(os.path.dirname(path), name_no), y=dp2d_crop_no.data, x=q_old)


# q_exp = dp2d.axes_manager.signal_axes[1].axis
# if q_exp.min() > crop_range_q[0] or q_exp.max() < crop_range_q[1]:
#     warnings.warn(
#         "The range at which signal was acquired is not large enough. Extrapolation will be used using scipy.interpolate.interp1d")
# q_new = np.linspace(crop_range_q[0], crop_range_q[1], crop_size)

# dp2d_crop = dp2d.map(interpolate_2d, q_array = q_exp, crop_range_q = crop_range_q, crop_size = crop_size,
#                     show_progressbar=True, parallel=True, inplace=False)

# # Correct for axes calibration
# sig_ax = dp2d_crop.axes_manager.signal_axes[1]
# sig_ax.offset = crop_range_q[0]
# sig_ax.scale = (crop_range_q[1] - crop_range_q[0]) / crop_size

# # Normalise data cropped for the npz
# dpmax = dp2d_crop.data.max((-2, -1), keepdims=True)
# dpmin = dp2d_crop.data.min((-2, -1), keepdims=True)
# dp_crop_dat = (dp2d_crop.data - dpmin) / (dpmax - dpmin)
# # Correct any nan value
# nan_mask = np.isnan(dp_crop_dat)
# dp_crop_dat[nan_mask] = 0

# # Save files
# name = "{}_crop2d_interpolated.hspy".format(os.path.basename(path).split('.')[0])
# dp2d_crop.save(os.path.join(os.path.dirname(path), name), overwrite=True)
# name = "{}_crop2d_interpolated.npz".format(os.path.basename(path).split('.')[0])
# np.savez(os.path.join(os.path.dirname(path), name), y=dp_crop_dat, x=q_new)



In [ ]:
# Plot results
f, axs = plt.subplots(ncols=2)

#axs[0].imshow(dp2d_crop.inav[:10,0].mean().data, vmax=10)
axs[1].imshow(dp2d_crop_no.inav[:10,0].mean().data, vmax=10)
axs[0].set_title("Interpolated mean")
axs[1].set_title("Non interpolated mean")
plt.tight_layout()

name_fig = "{}_fig.png".format(os.path.basename(path).split('.')[0])
plt.savefig(os.path.join(os.path.dirname(path), name_fig))

In [ ]:
dp2d_crop_no.plot(vmax=1)